In [7]:
# Cell 1 – Imports
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import numpy as np
import matplotlib.pyplot as plt

import pymc as pm
import arviz as az

# Suppress PyTensor g++ warning
import pytensor
pytensor.config.cxx = ""

# Optional fast sampler detection
try:
    import numpyro
    FAST_SAMPLER = "numpyro"
except ImportError:
    try:
        import blackjax
        FAST_SAMPLER = "blackjax"
    except ImportError:
        FAST_SAMPLER = None

print(f"Fast sampler: {FAST_SAMPLER or 'None (using default PyMC NUTS)'}")

Fast sampler: numpyro


In [8]:
# Cell 2 – Load & prepare data (expects focus period 2012+)
print("Loading data...")

preprocessor = BrentDataPreprocessor()
df_full = preprocessor.add_features(focus_period=None)
df = preprocessor.get_processed(focus_period='2012-01-01')

# Cell 2 – Prepare data arrays
prices = df['Price'].values.astype(np.float64)
dates  = df.index
N      = len(prices)

print(f"Modeling {N:,} observations from {dates.min():%Y-%m-%d} to {dates.max():%Y-%m-%d}")

Loading data...
Raw data loaded: 9011 rows, 2 columns
Data loaded successfully: 1987-05-20 to 2022-11-14
Total observations: 9011
Subset to >= 2012-01-01: 2760 observations remain
Modeling 2,760 observations from 2012-01-03 to 2022-11-14


In [10]:
# Cell 3 – Recommended version (automatic hybrid sampler)
print("Sampling with automatic hybrid NUTS + Metropolis...")

with pm.Model() as model:
    tau = pm.DiscreteUniform('tau', lower=0, upper=N-1)
    mu1 = pm.Normal('mu1', mu=prices.mean(), sigma=prices.std()*2.5)
    mu2 = pm.Normal('mu2', mu=prices.mean(), sigma=prices.std()*2.5)
    sigma = pm.HalfNormal('sigma', sigma=prices.std()*1.8)
    
    mu_t = pm.math.switch(tau >= np.arange(N), mu2, mu1)
    
    y = pm.Normal('y', mu=mu_t, sigma=sigma, observed=prices)

    trace = pm.sample(
        draws=600,
        tune=1000,
        chains=3,                # 3–4 is usually good balance
        target_accept=0.92,
        random_seed=42,
        return_inferencedata=True,
        progressbar=True,
        # Do NOT add nuts_sampler here — let PyMC choose automatically
    )

Sampling with automatic hybrid NUTS + Metropolis...


Multiprocess sampling (3 chains in 3 jobs)
CompoundStep
>Metropolis: [tau]
>NUTS: [mu1, mu2, sigma]
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


Output()

ValueError: Not enough samples to build a trace.

In [ ]:
# Cell 4 – Diagnostics
print(az.summary(trace, var_names=['tau', 'mu1', 'mu2', 'sigma'], hdi_prob=0.89))

az.plot_trace(trace, var_names=['tau', 'mu1', 'mu2'])
plt.tight_layout()
plt.show()

Posterior summary (89% HDI):


TypeError: Invalid format: 'hdi'. Focus options are: ('mean', 'median')